In [77]:
from pybaseball import playerid_lookup, statcast_single_game, statcast_batter, statcast_pitcher, playerid_reverse_lookup

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import requests
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from pybaseball.datasources.fangraphs import fg_pitching_data
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

def LorR(stand):
    if stand == 'R':
        return 0
    else: return 1
    
def pitch_type(pitch):
    #fastballs
    if pitch in ['FF', 'FC' , "FT" ,'FA' , 'SI' ,'SF']:
        return 0
    #changeups
    elif pitch in ["FS" ,'CH', 'KC']:
        return 1
    #curveballs
    elif pitch in ['KC' , 'CB' , "CU" , "KN"]:
        return 2
    
    #sliders
    elif pitch == 'SL': 
        return 3
    else: 
        return 4
    
def on_3b(b):
    if pd.isna(b):
        return 0
    else:
        return b
def count(balls,strikes):
    return "{ball}-{strike}".format(ball = balls, strike= strikes)
def cat_count(count):
    if count =='0-0':
        return 0
    elif count == '0-1':
        return 1
    elif count == '0-2':
        return 2
    elif count =='1-0':
        return 3
    elif count == '1-1':
        return 4
    elif count == '1-2':
        return 5
    elif count == '2-0':
        return 6
    elif count == '2-1':
        return 7
    elif count =='2-2':
        return 8
    elif count == '3-0':
        return 9
    elif count == '3-1':
        return 10
    elif count == '3-2':
        return 11


In [55]:
def getOnePitcher(pitcherID):
    pitches = statcast_pitcher('2000-04-01', '2022-12-31', player_id = pitcherID)
    pitches.dropna(subset=['pitch_type'],inplace=True)
    pitches = pitches.iloc[::-1]
    df_shifted = pitches.shift(1).add_prefix('previous_')
    result_df = pd.concat([pitches, df_shifted], axis=1 )
    result_df.reset_index(inplace= True)
    result_df.drop('index',axis = 1, inplace = True)
    result = copy.deepcopy(result_df)
    for i in range(len(result)):
        try:
            if(result['game_pk'][i]!=result['game_pk'][i-1]):
                result.drop(i,axis = 0, inplace = True)
        except:
            continue

    result['count'] = result.apply(lambda x:count(x.balls,x.strikes),axis =1)
    result['count'] = result['count'].apply(cat_count)
    training_df =result[['pitch_type','pitch_number','count','stand', 'previous_pitch_type']]
    training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))
    return training_df


In [53]:
def getAllPitchers(pitcherIDs, end_date):
    return_dict = {}
    for item in tqdm(pitcherIDs):
        return_dict[item] = getOnePitcher(item)
    return return_dict

In [38]:
def getIds(start,end):
    playernames = []
    players = list(fg_pitching_data(start,end)['Name'])
    for player in players:
        playernames.append(player.split(' '))
    ids = []
    for name in playernames:
        try:
            ids.append(int(playerid_lookup(name[1],name[0],True)["key_mlbam"]))
        except:
            continue
    return ids


In [47]:
ids = getIds(2015,2023)
print(len(ids))

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar

In [48]:
pitchers_to_add = []
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group A/Eddings 6-21-22/TOR at CWS 6-21-22.csv")['pitcher'])
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group A/Hallion 4-20-22/STL at MIA 4-20-22.csv")['pitcher'])
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group A/Nelson 4-16-22/OAK at TOR 4-16-22.csv")['pitcher'])
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group B/Hoberg 6-22-22/SFG at ATL 6-22-22.csv")['pitcher'])
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group B/Libka 9-12-22/TEX at MIA 9-12-22.csv")['pitcher'])
pitchers_to_add = pitchers_to_add + list(pd.read_csv("/Group B/Moscoso 9-16-22/NYY at MIL 9-16-22.csv")['pitcher'])





In [49]:
ids = ids+pitchers_to_add

In [50]:
ids = list(set(ids))

In [56]:
data_dict = getAllPitchers(ids, '2022-12-30')

  0%|          | 0/217 [00:00<?, ?it/s]

Gathering Player Data


/var/folders/1v/6kfsg1sj1h94v2jtztzln6rh0000gn/T/ipykernel_2728/130894328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))


Gathering Player Data


/Users/moksh/miniforge3/lib/python3.10/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/var/folders/1v/6kfsg1sj1h94v2jtztzln6rh0000gn/T/ipykernel_2728/130894328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))


...

/Users/moksh/miniforge3/lib/python3.10/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/var/folders/1v/6kfsg1sj1h94v2jtztzln6rh0000gn/T/ipykernel_2728/130894328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))


Gathering Player Data


/Users/moksh/miniforge3/lib/python3.10/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (0,43,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/Users/moksh/miniforge3/lib/python3.10/site-packages/pybaseball/utils.py:295: DtypeWarning: Columns (87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(data.text))
/var/folders/1v/6kfsg1sj1h94v2jtztzln6rh0000gn/T/ipykernel_2728/130894328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))


Gathering Player Data


/var/folders/1v/6kfsg1sj1h94v2jtztzln6rh0000gn/T/ipykernel_2728/130894328.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['stand'] = training_df['stand'].apply(lambda x:LorR(x))


In [57]:
df = pd.concat(data_dict.values(), ignore_index=True)
df.to_csv("/trainingpitches.csv")